# 位图

> 问题：有一千万个数字，范围在1～1亿之间。给你一个值，问这个值在这些数字中吗

> 一种用数组下标表示对应的数据，数组值1表示数据存在，0表示不存在的
>
> 例如，array[5]=1，表示数字5存在；array[5]=0， 表示数字5不存在
>
> 因为数组的元素类型是不确定的(python list)；即使使用array.array('B', []) 或者 Java 中的 int[]，每个元素占用的空间也是大于1bit的，因此我们需要一种紧凑的方式来表达位图
>
> 紧凑表示法的思路
> 1. 假如我们要存储的数据范围是 0 ～ 99
> 2. 用 array.array('B', []) 来表示位图
> 3. 计算数组大小，size = (99 >> 3) + 1，等价于 (100 // 8) + (1 if 100 & 7 > 0 else 0)
> 4. 计算数据的数组索引，byte_idx = n >> 3
> 5. 计算数据的位索引，bit_idx = n & 7
> 6. 添加数据 array[byte_idx] |= 1 << bit_idx
> 7. 删除数据 array[byte_idx] &= ~(1 << bit_idx)

> 回到开篇的问题，假设每个数字占4字节，如果用hash表存储这一千万个数字，需要40MB的存储空间，如果用位图存储，只需要12.5MB的存储空间
>
> 如果数据范围变成1～10亿呢？位图需要的存储空间变成125MB，反而比hash表更大了? 这时候该如何处理呢？

### 布隆过滤器

> 如果还是使用大小为1亿的位图，很明显1亿零1和1这两个数字会发生hash冲突
>
> 解决方法是，使用多个hash函数，将得到多个下标，同时处理这些下标对应的值。这样能有效的降低冲突的概率
>
> 所以布隆过滤器，对于数据存在的情况，存在小概率的误判，对于数据不存在的情况则没有问题，而且不支持删除操作

In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
import array

class BitSet:
    def __init__(self, n_bit):
        self.n = n_bit
        self.size = ((n_bit - 1) >> 3) + 1
        self.words = array.array('B', [0] * self.size)
    
    def __repr__(self):
        bin_str = [f'{w:08b}'[::-1] for w in self.words]
        if self.n & 7:
            bin_str[-1] = bin_str[-1][:self.n & 7]
        return ''.join(bin_str)


    def _index(self, bit):
        byte_idx = bit >> 3
        bit_idx = 1 << (bit & 7)
        return byte_idx, bit_idx

    def set(self, bit):
        if bit < 0 or bit >= self.n:
            raise Exception('out of index range')
        
        byte_idx, bit_idx = self._index(bit)
        self.words[byte_idx] |= bit_idx
    
    def clear(self, bit):
        if bit < 0 or bit >= self.n:
            raise Exception('out of index range')
        
        byte_idx, bit_idx = self._index(bit)
        self.words[byte_idx] &= ~bit_idx

bit_set = BitSet(10)
bit_set.set(3)
bit_set.set(8)
bit_set
bit_set.clear(4)
bit_set
bit_set.clear(3)
bit_set.clear(3)
bit_set


0001000010

0001000010

0000000010